# Spotify Sound Guide by Sergey Sonkin

The code below is broken up into steps for 
* Step 0: Deal with env and global variables, including the artist's spotify id
* Step 1: Get artist's albums
* Step 2: Filter out duplicate albums
* Step 3: Get track information, engineer any new features
* Step 4: Repeat steps 1-3 but for artist's singles (done separately from albums to avoid duplicates)
* Step 5: Process features for recommendation
* Step 6: Get a seed song
* Step 7: Define our likeability metric
* Step 8: Start recommending!

## Step 0: Dealing with request variables

In [145]:
import os
import requests
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

data = "grant_type=client_credentials&client_id=" \
        + client_id \
        + "&client_secret=" \
        + client_secret
header = {
    "Content-Type": "application/x-www-form-urlencoded"
}
response = requests.post("https://accounts.spotify.com/api/token",headers=header,data=data)
access_token = response.json()['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}
BASE_URL = 'https://api.spotify.com/v1/'

## Some artist ids
global_artist_id = "6qqNVTkY8uBg9cP3Jd7DAH" ##Billie Eilish
global_artist_id = "6fWVd57NKTalqvmjRd2t8Z" ##24kGoldn
global_artist_id = "2tIP7SsRs7vjIcLrU85W8J" ##The Kid Laroi
global_artist_id = "7dGJo4pcD2V6oG8kP0tJRR" ##Eminem
global_artist_id = "6wWVKhxIU2cEi0K81v7HvP" ##Rammstein
global_artist_id = "2kRfqPViCqYdSGhYSM9R0Q" ##Madison Beer

## Step 1: Start by getting just albums

In [146]:
def get_albums(type='album',artist_id=global_artist_id):
    album_list = []
    counter = 0 ## Need counter to deal with limit of 50 per page
    while(True):
        print(counter)
        r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                        headers=headers, 
                        params={'market':'US', 'include_groups': type, 'limit': 50, 'offset': 50*counter})
        d = r.json()
        if len(d['items']) == 0:
            print("Done!")
            break
        album_list += d['items']
        counter += 1
    return album_list

album_list = get_albums('album')

0
1
Done!


## Step 2: Filter out duplicate albums

How do we pick which of the duplicates to use? We choose based on the following criteria in order

1. Most explicit
2. Most recently released
3. Most number of tracks

In [147]:
def filter_duplicates(album_list):
    ## Generating list of album names
    names = [(i,re.sub(r'\W+', '',album['name'].lower())) for i,album in enumerate(album_list)]
    ## Generating list of duplicates (doubles)
    viewed = {}
    doubles = []
    for (index,name) in names:
        if name in viewed:
            other_index = viewed[name]
            doubles.append((name,index,other_index))
        else:
            viewed[name] = index

    ## How do we pick which of the dupli
    ## For each duplicate album, find which one has the explicit songs
    for (name,index_1,index_2) in doubles:
        album_id_1 = album_list[index_1]['id']
        album_id_2 = album_list[index_2]['id']
        r1 = requests.get(BASE_URL+'albums/'+album_id_1+'/tracks',headers=headers)
        r2 = requests.get(BASE_URL+'albums/'+album_id_2+'/tracks',headers=headers)
        tracks1 = r1.json()['items']
        tracks2 = r2.json()['items']
        explicit_1 = tracks1[0]['explicit']
        explicit_2 = tracks2[0]['explicit']
        ## If one is explicit but not the other, take the explicit version
        if explicit_1 and not explicit_2:
            viewed[name] = index_1
        elif explicit_2 and not explicit_1:
            viewed[name] = index_2
        ## If they're the same explicitness, take the more recently released version
        else:
            album_rd_1 = album_list[index_1].get("release_date",-1)
            album_rd_2 = album_list[index_2].get("release_date",-1)
            if album_rd_1 > album_rd_2:
                viewed[name] = index_1
            elif album_rd_2 > album_rd_1:
                viewed[name] = index_2
            else:
            ## If they're release on the same date, take the one with more tracks
                album_tracks_1 = album_list[index_1].get("total_tracks",-1)
                album_tracks_2 = album_list[index_2].get("total_tracks",-1)
                if album_tracks_1 > album_tracks_2:
                    viewed[name] = index_1
                elif album_tracks_2 > album_tracks_1:
                    viewed[name] = index_2
                ## If they have the same number of tracks honestly I'm defeated just pick the first
                else:
                    viewed[name] = index_1
    filtered_album_ids = list(viewed.values())
    print("We removed " + str(len(album_list) - len(filtered_album_ids)) + " duplicates")
    return filtered_album_ids

filtered_album_ids = filter_duplicates(album_list)

We removed 0 duplicates


## Step 3: Get track information

The star of the show is Spotify's "Get Tracks' Audio Features" endpoint. Its documention can be found here https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features

### Step 3a: Initialize track information

First we create our pandas table with the basic information about each track we have from our queries above. 

If we have duplicate tracks, we filter for them as we go.

In [148]:
track_info = pd.DataFrame(columns = ["album_name","track_name","release_date","popularity","duration_ms",
                                     "acousticness","danceability","energy","instrumentalness","key","liveness",
                                     "loudness","mode","speechiness","tempo","time_signature","valence"])
track_info[['mode', 'time_signature']] = track_info[['mode', 'time_signature']].astype('int8')

## We will store track ids in a list for use in Step 3b
track_names = set()

def get_new_tracks(album_list,filtered_album_ids):
    global track_info
    track_ids = []
    for album_id in filtered_album_ids:
        ## Get the album
        album = album_list[album_id]
        album_name = album["name"]
        album_release_date = album["release_date"]
        ## Get the tracks for that album
        id = album['id']
        r = requests.get(BASE_URL+'albums/'+id+'/tracks',headers=headers)
        tracks = r.json()['items']
        ## For each track, just get the track name and id and set some initial values
        for track in tracks:
            track_id = track['id']
            track_name = track['name']
            track_popularity = track.get('popularity',0.0) ## Default value of 0
            duration_ms = track.get('duration_ms',180000) ## Default value of 3 minutes
            if track_name not in track_names:
                ## Creating new DF with desired column names
                new_track_dict = {"album_name":album_name,
                                  "track_name":track_name,
                                  "release_date":pd.to_datetime(album_release_date),
                                  "popularity":track_popularity,
                                  "duration_ms":duration_ms,
                                  "acousticness":0.0,
                                  "danceability":0.0,
                                  "energy":0.0,
                                  "instrumentalness":0.0,
                                  "key":0.0,
                                  "liveness":0.0,
                                  "loudness":0.0,
                                  "mode":0,
                                  "speechiness":0.0,
                                  "tempo":0.0,
                                  "time_signature":0,
                                  "valence":0.0}
                new_track_row = pd.DataFrame([new_track_dict],index=[track_id])
                ## Append to track_info, track_ids, and track_names
                track_info = track_info.append(new_track_row)
                track_ids.append(track_id)
                track_names.add(track_name)
            else:
                print("We have a duplicate track name: " + track_name)
    return track_ids

track_ids = get_new_tracks(album_list,filtered_album_ids)

### Step 3b: Get song details

Now for all of the (unique) tracks from 3a, we ask Spotify for its song features.

Every once in a while Spotify just doesn't have any song features so we drop the song completely. It's not helpful to us if we don't know anything about it, and it happens so rarely that it's not a concern for the usability of this project.

In [149]:
special_features = ["acousticness","danceability","energy","instrumentalness","key","liveness",
                    "loudness","mode","speechiness","tempo","time_signature","valence"]
def get_features(track_ids,track_info):
    ## Break our song IDs into 100 song batches
    l = len(track_ids)
    width = 100
    iters = (l // width) + 1
    for ii in range(iters):
        ## Get these 100 songs intro a string
        track_id_subset = track_ids[width*ii:width*ii+width]
        tracks_string = (",").join(track_id_subset)
        ## Get the request with this data
        r = requests.get(BASE_URL + 'audio-features/',headers=headers,
                        params={'ids':tracks_string})
        audio_features = r.json()['audio_features']
        ## We got the audio features, now extract as much as possible
        for jj,audio_feature in enumerate(audio_features):
            track_id = track_id_subset[jj]
            try:
                for f in special_features:
                    track_info.loc[track_id,f] = audio_feature[f]
            except:
                track_id = jj + width*ii
                print("We have an issue with track id " + str(track_id) + " " + str(track_ids[track_id]))
                track_info = track_info.drop(track_ids[track_id])


get_features(track_ids,track_info)

In [150]:
track_info

,album_name,track_name,release_date,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0lhDiOviH4xmzyYeOsn01M,Life Support,The Beginning,2021-02-26,0.0,58239,0.4390,0.262,0.301,0.002410,11.0,0.0748,-9.702,1,0.0335,102.399,3,0.0481
4y3g40TWe7fCWCayJZuGvw,Life Support,Good In Goodbye,2021-02-26,0.0,141949,0.4330,0.658,0.698,0.000000,11.0,0.1740,-5.950,0,0.1770,139.054,4,0.4560
5AEJyckUMzLqIyBxZNmGFW,Life Support,Default,2021-02-26,0.0,116741,0.5560,0.143,0.327,0.000657,9.0,0.4440,-11.204,0,0.0327,187.194,4,0.2400
2Txj791OlOaNSMpps0gz5K,Life Support,Follow The White Rabbit,2021-02-26,0.0,179712,0.2220,0.650,0.405,0.000000,2.0,0.0868,-6.023,1,0.0329,95.005,4,0.2200
6sRmaXQZo0EK2woVn2COxM,Life Support,Effortlessly,2021-02-26,0.0,168655,0.5840,0.309,0.567,0.000004,0.0,0.0681,-6.768,0,0.0498,75.276,4,0.2670
6mstsFlnLZidRE8K3JGtV7,Life Support,Stay Numb And Carry On,2021-02-26,0.0,163548,0.2130,0.611,0.580,0.000000,7.0,0.2200,-6.455,0,0.0355,92.489,4,0.1730
4a1RWaG4BTkifgMSx3rpf3,Life Support,Blue,2021-02-26,0.0,229881,0.1500,0.640,0.501,0.000000,7.0,0.1600,-7.860,1,0.0624,114.031,4,0.3460
3Fsr7IhlNBZ1wV3QaJB5dv,Life Support,Interlude,2021-02-26,0.0,109647,0.7060,0.390,0.176,0.000000,0.0,0.1260,-11.678,1,0.0485,90.857,5,0.2980
5QlMftmj7rFwhHoClCQLTP,Life Support,Homesick,2021-02-26,0.0,227197,0.5460,0.332,0.416,0.000000,0.0,0.0930,-7.111,1,0.0326,81.198,4,0.3300
5UY8jf31X2hJkiAualFTyh,Life Support,Selfish,2021-02-26,0.0,223270,0.6270,0.375,0.461,0.000000,9.0,0.3860,-6.202,1,0.0279,75.217,4,0.2330


In [151]:
## Just outputting to a CSV so I can experiment in tableau
track_info.to_csv("out2.csv")

## Step 4: Repeat Steps 1-3 but for singles

We already put steps 1-3 into functions so this doesn't require much duplicate code at all!

In the future, instead of calling get_albums on albums and singles individually we could call for both at the same time saving on at most 1 API call per artist. 

If we go into large scale production and get bottlenecked by rate limits, this could be a good improvement. For now, it's pedantic and not worth the energy to rewrite otherwise working code.

### Step 4a: Get singles

In [152]:
single_list = get_albums('single')

0
1
Done!


### Step 4b: Filter duplicate singles

In [153]:
filtered_single_ids = filter_duplicates(single_list)

We removed 0 duplicates


### Step 4c: Retrieve singles and add if they're unique

In [154]:
track_ids = get_new_tracks(single_list,filtered_single_ids)

We have a duplicate track name: BOYSHIT
We have a duplicate track name: Baby
We have a duplicate track name: Stained Glass
We have a duplicate track name: Selfish
We have a duplicate track name: Good In Goodbye
We have a duplicate track name: Say It to My Face
We have a duplicate track name: Dead


### Step 4d: Retrieve features for these new singles

In [155]:
get_features(track_ids,track_info)

In [156]:
track_info

,album_name,track_name,release_date,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0lhDiOviH4xmzyYeOsn01M,Life Support,The Beginning,2021-02-26,0.0,58239,0.43900,0.262,0.301,0.002410,11.0,0.0748,-9.702,1,0.0335,102.399,3,0.0481
4y3g40TWe7fCWCayJZuGvw,Life Support,Good In Goodbye,2021-02-26,0.0,141949,0.43300,0.658,0.698,0.000000,11.0,0.1740,-5.950,0,0.1770,139.054,4,0.4560
5AEJyckUMzLqIyBxZNmGFW,Life Support,Default,2021-02-26,0.0,116741,0.55600,0.143,0.327,0.000657,9.0,0.4440,-11.204,0,0.0327,187.194,4,0.2400
2Txj791OlOaNSMpps0gz5K,Life Support,Follow The White Rabbit,2021-02-26,0.0,179712,0.22200,0.650,0.405,0.000000,2.0,0.0868,-6.023,1,0.0329,95.005,4,0.2200
6sRmaXQZo0EK2woVn2COxM,Life Support,Effortlessly,2021-02-26,0.0,168655,0.58400,0.309,0.567,0.000004,0.0,0.0681,-6.768,0,0.0498,75.276,4,0.2670
6mstsFlnLZidRE8K3JGtV7,Life Support,Stay Numb And Carry On,2021-02-26,0.0,163548,0.21300,0.611,0.580,0.000000,7.0,0.2200,-6.455,0,0.0355,92.489,4,0.1730
4a1RWaG4BTkifgMSx3rpf3,Life Support,Blue,2021-02-26,0.0,229881,0.15000,0.640,0.501,0.000000,7.0,0.1600,-7.860,1,0.0624,114.031,4,0.3460
3Fsr7IhlNBZ1wV3QaJB5dv,Life Support,Interlude,2021-02-26,0.0,109647,0.70600,0.390,0.176,0.000000,0.0,0.1260,-11.678,1,0.0485,90.857,5,0.2980
5QlMftmj7rFwhHoClCQLTP,Life Support,Homesick,2021-02-26,0.0,227197,0.54600,0.332,0.416,0.000000,0.0,0.0930,-7.111,1,0.0326,81.198,4,0.3300
5UY8jf31X2hJkiAualFTyh,Life Support,Selfish,2021-02-26,0.0,223270,0.62700,0.375,0.461,0.000000,9.0,0.3860,-6.202,1,0.0279,75.217,4,0.2330


### Step 5: Clean and process our data for recommendation

A couple of things to point out. 

#### Onehot encoding

We need to onehot encode our data because our similarity metric uses cosine similarity/distance, which does not work with categorical columns. 

Some classes that could be encoded but probably shouldn't be: Keys, Pitch Classes, Time Signature. 

These features (from Spotify) each have their own basis in numeric space (i.e. distance between Pitch Class 2 and 3 vs 2 and 4 does have mathematical significance) so we leave it as is.

#### Dealing with release date

We could convert release date into Unix time in seconds but I figured a more practical approach would be to instead look at the artists "progress" into their career: whether a song was released at the start or towards their latest release.

Artists, now more than ever, tend to develop and change their sound over time especially with every major album release. Much of this could be attributed to the change from buying songs in records to browsing songs on streaming services but that's a separate rant.

By converting release date into a range of [0,1] we are mapping songs together to account for an artist's growth while still also considering album groupings because album_name is being onehot encoded as well.

Arguably this is overfitting, but in testing it almost seems to not overfit enough. 

A large reason for this is because if an artist only started recently, the difference between 0 and 1 is huge, but the difference between 2022 and 2023 isn't as an artist isn't likely to change their sound so drastically in a year. 

In the future, one way to deal with this would be to creative a "minimum" of five years to every artist's career so that even if they have only been producing for a year, their release dates get mapped to 0.9 and 1 so they're not as far apart. Currently not needed, but I'm documenting this for myself in case I need to revisit this idea.

### Removing interludes

Often artists will include interludes in their albums or joke tracks or other songs not worth recommending. These tracks aren't flagged by Spotify in any particular way. We will attempt to find these tracks ourselves by finding extreme statistical outliers using the IQR for song duration.

Interludes will get suggested so rarely that it's significantly more important that we don't over-remove songs than we do remove all interludes.

One suggestion would be to set a song minimum of 30 seconds, but there could be a case where someone's favorite Spotify artist uploads 25 seconds spoken word and we wouldn't want to just delete all of their tracks. That's why we chose to go with the IQR approach with a number of 3.6 instead of the standard 1.5. 

For a typical artist this approach will map to around 30 seconds anyways while accounting for the edge cases.

In [157]:
start = 0

def clean_data(track_info):
    ## In theory we could also get dummies for keys but pitch classes seem to have numeric analogue
    ## Same goes for time_signature
    cleaned = pd.get_dummies(track_info,columns=['album_name','mode'])

    ## Converting release date to a "career" field
    start = track_info['release_date'].min()
    cleaned['career'] = track_info['release_date'] - start
    cleaned['career'] = cleaned['career'].dt.days
    end = cleaned['career'].max()
    cleaned['career'] = cleaned['career'] / end

    ## Finding duration outliers and dropping them!!
    durs = cleaned["duration_ms"]
    Q1 = durs.quantile(0.25)
    Q3 = durs.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3.6*IQR
    keep = np.array(durs>lower_bound)
    removed_cnt = np.size(keep) - np.count_nonzero(keep)
    cleaned = cleaned.loc[keep]
    if removed_cnt > 0:
        print("We removed " + str(removed_cnt) + " songs based on length")

    ## CHanging loudness from [min,max] to [0,1]
    cleaned["loudness"] = cleaned["loudness"] - cleaned["loudness"].min()
    cleaned["loudness"] = cleaned["loudness"] / cleaned["loudness"].max()

    ## Drop all columns we don't care about anymore
    # cleaned = cleaned.drop(columns=['track_name','popularity','release_date','duration_ms'])
    cleaned = cleaned.drop(columns=['track_name','release_date','duration_ms'])
    return cleaned

cleaned = clean_data(track_info)

In [158]:
cleaned

,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,...,album_name_Say It to My Face (The Wideboys Remix),album_name_Selfish (Alan Walker Remix),album_name_Showed Me (How I Fell In Love With You),album_name_Something Sweet,album_name_Something Sweet (Exit Friendzone Remix),album_name_Unbreakable,album_name_Unbreakable (Monsieur Adi Remix),mode_0,mode_1,career
0lhDiOviH4xmzyYeOsn01M,0.0,0.43900,0.262,0.301,0.002410,11.0,0.0748,0.208835,0.0335,102.399,...,0,0,0,0,0,0,0,0,1,0.833707
4y3g40TWe7fCWCayJZuGvw,0.0,0.43300,0.658,0.698,0.000000,11.0,0.1740,0.605369,0.1770,139.054,...,0,0,0,0,0,0,0,1,0,0.833707
5AEJyckUMzLqIyBxZNmGFW,0.0,0.55600,0.143,0.327,0.000657,9.0,0.4440,0.050095,0.0327,187.194,...,0,0,0,0,0,0,0,1,0,0.833707
2Txj791OlOaNSMpps0gz5K,0.0,0.22200,0.650,0.405,0.000000,2.0,0.0868,0.597654,0.0329,95.005,...,0,0,0,0,0,0,0,0,1,0.833707
6sRmaXQZo0EK2woVn2COxM,0.0,0.58400,0.309,0.567,0.000004,0.0,0.0681,0.518918,0.0498,75.276,...,0,0,0,0,0,0,0,1,0,0.833707
6mstsFlnLZidRE8K3JGtV7,0.0,0.21300,0.611,0.580,0.000000,7.0,0.2200,0.551997,0.0355,92.489,...,0,0,0,0,0,0,0,1,0,0.833707
4a1RWaG4BTkifgMSx3rpf3,0.0,0.15000,0.640,0.501,0.000000,7.0,0.1600,0.403509,0.0624,114.031,...,0,0,0,0,0,0,0,0,1,0.833707
3Fsr7IhlNBZ1wV3QaJB5dv,0.0,0.70600,0.390,0.176,0.000000,0.0,0.1260,0.000000,0.0485,90.857,...,0,0,0,0,0,0,0,0,1,0.833707
5QlMftmj7rFwhHoClCQLTP,0.0,0.54600,0.332,0.416,0.000000,0.0,0.0930,0.482668,0.0326,81.198,...,0,0,0,0,0,0,0,0,1,0.833707
5UY8jf31X2hJkiAualFTyh,0.0,0.62700,0.375,0.461,0.000000,9.0,0.3860,0.578736,0.0279,75.217,...,0,0,0,0,0,0,0,0,1,0.833707


## Step 6: Get a seed song and define our likeability metric

How do we pick the first song to recommend? Picking the most popular one is probably a safe bet - it's popular for a reason!

### Step 6a: Get most popular song for seed song

This code will break if none of their most popular songs are in our database.

This would only happen if all of their most popular songs are by another artist, which I haven't seen yet but let's be honest if that's the case you probably shouldn't listen to them anyways.

It's really easy to just pick a random seed song instead, but I think this approach is significantly funnier.

In [159]:
## Get most popular tracks
r = requests.get(BASE_URL + 'artists/' + global_artist_id + '/top-tracks',headers=headers,params={'market':'US'})
trracks = r.json()['tracks']
still_searching = True
## As we iterate over tracks, get their popularity metrics
for track in trracks:
    most_id = track["id"]
    ## Record the popularity IF it's in the index
    if most_id in cleaned.index:
        pop = track.get("popularity",0)
        cleaned.loc[most_id,"popularity"] = pop/100
        ## If we still need a seed song, write it down
        if still_searching:
            most_pop = cleaned.loc[most_id].to_frame().transpose()
            still_searching = False
if still_searching:
    raise Exception("This artist stinks we couldn't find a single song of theirs that was popular to suggest")
most_pop

,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,...,album_name_Say It to My Face (The Wideboys Remix),album_name_Selfish (Alan Walker Remix),album_name_Showed Me (How I Fell In Love With You),album_name_Something Sweet,album_name_Something Sweet (Exit Friendzone Remix),album_name_Unbreakable,album_name_Unbreakable (Monsieur Adi Remix),mode_0,mode_1,career
5ajjAnNRh8bxFvaVHzpPjh,0.81,0.807,0.386,0.426,0.0,3.0,0.14,0.532234,0.0363,180.104,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.861142


### Step 7: Start computing distances

I'll create a video explaining why we're using this metric.

In [160]:
(r,_) = track_info.shape
prev = np.ones((r,1))

## Store track_ids of songs already recommended
viewed_tracks = []

def get_song_recommendation(stripped,new_track_info,liked=True):
    global prev
    ## Step 0: Mark this track as viewed
    prev_id = new_track_info.index[0]
    viewed_tracks.append(prev_id)
    ## Step 1: Get distances from new track
    dists = cosine_similarity(stripped,new_track_info)
    ## Step 2: If we hated that song, we want songs that are far away from it
    if not liked:
        dists = 1 - dists
    ## Step 3: Get new metric array
    prev = np.multiply(prev,dists)
    ## Step 4: From this array, get next song to recommend
    sortedd = np.argsort(prev,axis=0)
    for ii in range(1,20):
        jj = sortedd[-ii][0]
        new_track_id = stripped.iloc[jj].name
        if new_track_id not in viewed_tracks:
            return stripped.iloc[jj].to_frame().transpose()

### Step 8: Start recommending songs and hearing back!

In [161]:
from time import sleep

new_rec = most_pop
for _ in range(5):
    new_rec_id = new_rec.index[0]
    new_rec_name = track_info.loc[new_rec_id,"track_name"]
    print("You should check out " + new_rec_name + " (spotify:track:" + new_rec_id + ")")
    sleep(1)
    enjoyed = input("Did you enjoy this track? Yes or No")
    print("Did you enjoy this track? " + enjoyed)
    if enjoyed == "Yes":
        new_rec = get_song_recommendation(cleaned,new_rec,True)
    else:
        new_rec = get_song_recommendation(cleaned,new_rec,False)

You should check out Reckless (spotify:track:5ajjAnNRh8bxFvaVHzpPjh)
Did you enjoy this track? Yes
You should check out Carried Away (Love To Love) (with Madison Beer) (spotify:track:4IvuPZogXbY7LODs7qzr0W)
Did you enjoy this track? Yes
You should check out Sour Times (spotify:track:4wz7R8N7gYgjhZuoMQkSve)
Did you enjoy this track? Yes
You should check out BOYSHIT (spotify:track:6XiqdnMk2LLbmybmyx30jD)
Did you enjoy this track? Yes
You should check out MORE (spotify:track:6juLaduD4STCUDWT0AYun4)
Did you enjoy this track? No
